In [1]:
# Feature Generation Notebook

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/all_timeline.csv')

In [3]:
df.columns

Index(['date', 'countrycode', 'countrylabel', 'showcountrylabelcases',
       'labelpositioncases', 'showcountrylabeldeaths', 'labelpositiondeaths',
       'showcountrylabelrecovered', 'labelpositionrecovered', 'casesoverride',
       'deathsoverride', 'recoveredoverride',
       'columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit',
       'emergencycasesoverride', 'emergencydeathsoverride', 'totalcases',
       'totaldeaths', 'totalrecovered'],
      dtype='object')

In [4]:
df.head(5)

,date,countrycode,countrylabel,showcountrylabelcases,labelpositioncases,showcountrylabeldeaths,labelpositiondeaths,showcountrylabelrecovered,labelpositionrecovered,casesoverride,deathsoverride,recoveredoverride,columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit,emergencycasesoverride,emergencydeathsoverride,totalcases,totaldeaths,totalrecovered
0,1/22/20,AF,Afghanistan,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/22/20,AL,Albania,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,THIS SPREADSHEET IS THE OLD VERSION. DO NOT US...,NaN,NaN,NaN,NaN,NaN
2,1/22/20,DZ,Algeria,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,TO UPDATE UK NUMBER USE THIS NEW SPREADSHEET P...,NaN,NaN,NaN,NaN,NaN
3,1/22/20,AO,Angola,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/22/20,AR,Argentina,no,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Remove Dummy Columns
del df['labelpositioncases']
del df['showcountrylabeldeaths']
del df['labelpositiondeaths']
del df['showcountrylabelrecovered']
del df['labelpositionrecovered']
del df['showcountrylabelcases']
del df['columnsonrightshouldbeemptywhennotinuseifyouenteranumberitwillalwaysshowunlessyoudeleteit']

del df['casesoverride']
del df['deathsoverride']
del df['recoveredoverride']
del df['emergencycasesoverride']
del df['emergencydeathsoverride']

In [6]:
# Remove Rows with totalcases == Nan
df = df[df['totalcases'].notna()]

# Update totaldeaths and totalrecovered columns to 0 if is Nan
df.loc[:, ('totaldeaths')] = df.apply(
    lambda row: row['totaldeaths'] if not np.isnan(row['totaldeaths']) else 0.0, axis=1
)

df.loc[:, ('totalrecovered')] = df.apply(
    lambda row: row['totalrecovered'] if not np.isnan(row['totalrecovered']) else 0.0, axis=1
)

df = df[df['countrycode'] == 'BR']

df.head(100)

,date,countrycode,countrylabel,totalcases,totaldeaths,totalrecovered
6284,2/26/20,BR,Brazil,1.0,0.0,0.0
6463,2/27/20,BR,Brazil,1.0,0.0,0.0
6642,2/28/20,BR,Brazil,1.0,0.0,0.0
6821,2/29/20,BR,Brazil,2.0,0.0,0.0
7000,3/1/20,BR,Brazil,2.0,0.0,0.0
7179,3/2/20,BR,Brazil,2.0,0.0,0.0
7358,3/3/20,BR,Brazil,2.0,0.0,0.0
7537,3/4/20,BR,Brazil,4.0,0.0,0.0
7716,3/5/20,BR,Brazil,4.0,0.0,0.0
7895,3/6/20,BR,Brazil,13.0,0.0,0.0


In [7]:
# Create all New Columns
df.loc[:, ('ft01_previous_day_totalcases')] = 0
df.loc[:, ('ft02_previous_day_totaldeaths')] = 0
df.loc[:, ('ft03_previous_day_totalrecovered')] = 0

df.loc[:, ('ft04_new_cases_per_day')] = 0
df.loc[:, ('ft05_new_deaths_per_day')] = 0
df.loc[:, ('ft06_new_recovered_per_day')] = 0

df.loc[:, ('ft07_new_cases_evolution_rate')] = 0
df.loc[:, ('ft08_new_deaths_evolution_rate')] = 0
df.loc[:, ('ft09_new_recovered_evolution_rate')] = 0

In [8]:
# Export all Columns Indexes
totalcases_column_ix = df.columns.get_loc('totalcases')
totaldeaths_column_ix = df.columns.get_loc('totaldeaths')
totalrecovered_column_ix = df.columns.get_loc('totalrecovered')

ft01_previous_day_totalcases_column_ix = df.columns.get_loc('ft01_previous_day_totalcases')
ft02_previous_day_totaldeaths_column_ix = df.columns.get_loc('ft02_previous_day_totaldeaths')
ft03_previous_day_totalrecovered_column_ix = df.columns.get_loc('ft03_previous_day_totalrecovered')

ft04_new_cases_per_day_columns_ix = df.columns.get_loc('ft04_new_cases_per_day')
ft05_new_deaths_per_day_columns_ix = df.columns.get_loc('ft05_new_deaths_per_day')
ft06_new_recovered_per_day_columns_ix = df.columns.get_loc('ft06_new_recovered_per_day')

ft07_new_cases_evolution_rate_columns_ix = df.columns.get_loc('ft07_new_cases_evolution_rate')
ft08_new_deaths_evolution_rate_columns_ix = df.columns.get_loc('ft08_new_deaths_evolution_rate')
ft09_new_recovered_evolution_rate_columns_ix = df.columns.get_loc('ft09_new_recovered_evolution_rate')

In [ ]:
# Feature 01 - Previous Day Total Cases Count
for i in range(1, len(df.index)):
    df.iloc[i, ft01_previous_day_totalcases_column_ix] = df.iloc[i-1, totalcases_column_ix]

In [ ]:
# Feature 02 - Previous Day Total Deaths Count
for i in range(1, len(df.index)):
    df.iloc[i, ft02_previous_day_totaldeaths_column_ix] = df.iloc[i-1, totaldeaths_column_ix]

In [ ]:
# Feature 03 - Previous Day Total Recovered Count
for i in range(1, len(df.index)):
    df.iloc[i, ft03_previous_day_totalrecovered_column_ix] = df.iloc[i-1, totalrecovered_column_ix]

In [ ]:
# Feature 04 - New Cases per Day
df.iloc[:, ft04_new_cases_per_day_columns_ix] = \
    df.iloc[:, totalcases_column_ix] - df.iloc[:, ft01_previous_day_totalcases_column_ix]

In [ ]:
# Feature 05 - New Deaths per Day
df.iloc[:, ft05_new_deaths_per_day_columns_ix] = \
    df.iloc[:, totaldeaths_column_ix] - df.iloc[:, ft02_previous_day_totaldeaths_column_ix]

In [ ]:
# Feature 06 - New Recovered per Day
df.iloc[:, ft06_new_recovered_per_day_columns_ix] = \
    df.iloc[:, totalrecovered_column_ix] - df.iloc[:, ft03_previous_day_totalrecovered_column_ix]

In [ ]:
# As 3 features abaixo nao estao sendo calculadas direito. Eu quero o Rate (taxa) da Mudanca

In [ ]:
# Feature 07 - New Cases Evolution Rate
df.iloc[:, ft07_new_cases_evolution_rate_columns_ix] = \
    df.iloc[:, totalcases_column_ix] / df.iloc[:, ft01_previous_day_totalcases_column_ix]

In [ ]:
# Feature 08 - New Deaths Evolution Rate
df.iloc[:, ft08_new_deaths_evolution_rate_columns_ix] = \
    df.iloc[:, totaldeaths_column_ix] / df.iloc[:, ft02_previous_day_totaldeaths_column_ix]

In [ ]:
# Feature 09 - New Recovered Evolution Rate
df.iloc[:, ft09_new_recovered_evolution_rate_columns_ix] = \
    df.iloc[:, totalrecovered_column_ix] / df.iloc[:, ft03_previous_day_totalrecovered_column_ix]

In [ ]:
df.head(100)

In [9]:
# Compute Death Evolution per Day
target_country.loc[:, ('death_rate_previous_day')] = 0
death_rate_previous_day_column_ix = target_country.columns.get_loc('death_rate_previous_day')
totaldeaths_ix = target_country.columns.get_loc('totaldeaths')

for i in range(1, len(target_country.index)):
    target_country.iloc[i, death_rate_previous_day_column_ix] = \
        (target_country.iloc[i, totaldeaths_ix] / target_country.iloc[i-1, totaldeaths_ix])-1

In [10]:
# Feature 02 - Previous Day Total Deaths Count
for i in range(1, len(df.index)):
    df.iloc[i, ft02_previous_day_totaldeaths_column_ix] = df.iloc[i-1, totaldeaths_column_ix]

In [11]:
# Feature 03 - Previous Day Total Recovered Count
for i in range(1, len(df.index)):
    df.iloc[i, ft03_previous_day_totalrecovered_column_ix] = df.iloc[i-1, totalrecovered_column_ix]

In [12]:
# Feature 04 - New Cases per Day
df.iloc[:, ft04_new_cases_per_day_columns_ix] = \
    df.iloc[:, totalcases_column_ix] - df.iloc[:, ft01_previous_day_totalcases_column_ix]

In [13]:
# Feature 05 - New Deaths per Day
df.iloc[:, ft05_new_deaths_per_day_columns_ix] = \
    df.iloc[:, totaldeaths_column_ix] - df.iloc[:, ft02_previous_day_totaldeaths_column_ix]

In [14]:
# Feature 06 - New Recovered per Day
df.iloc[:, ft06_new_recovered_per_day_columns_ix] = \
    df.iloc[:, totalrecovered_column_ix] - df.iloc[:, ft03_previous_day_totalrecovered_column_ix]

In [ ]:
# As 3 features abaixo nao estao sendo calculadas direito. Eu quero o Rate (taxa) da Mudanca

In [22]:
# Feature 07 - New Cases Evolution Rate
df.iloc[:, ft07_new_cases_evolution_rate_columns_ix] = \
    df.iloc[:, totalcases_column_ix] / df.iloc[:, ft01_previous_day_totalcases_column_ix]

In [16]:
# Feature 08 - New Deaths Evolution Rate
df.iloc[:, ft08_new_deaths_evolution_rate_columns_ix] = \
    df.iloc[:, totaldeaths_column_ix] / df.iloc[:, ft02_previous_day_totaldeaths_column_ix]

In [17]:
# Feature 09 - New Recovered Evolution Rate
df.iloc[:, ft09_new_recovered_evolution_rate_columns_ix] = \
    df.iloc[:, totalrecovered_column_ix] / df.iloc[:, ft03_previous_day_totalrecovered_column_ix]

In [23]:
df.head(100)

,date,countrycode,countrylabel,totalcases,totaldeaths,totalrecovered,ft01_previous_day_totalcases,ft02_previous_day_totaldeaths,ft03_previous_day_totalrecovered,ft04_new_cases_per_day,ft05_new_deaths_per_day,ft06_new_recovered_per_day,ft07_new_cases_evolution_rate,ft08_new_deaths_evolution_rate,ft09_new_recovered_evolution_rate
6284,2/26/20,BR,Brazil,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,inf,NaN,NaN
6463,2/27/20,BR,Brazil,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN
6642,2/28/20,BR,Brazil,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN
6821,2/29/20,BR,Brazil,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.000000,NaN,NaN
7000,3/1/20,BR,Brazil,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN
7179,3/2/20,BR,Brazil,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN
7358,3/3/20,BR,Brazil,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN
7537,3/4/20,BR,Brazil,4.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.000000,NaN,NaN
7716,3/5/20,BR,Brazil,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaN
7895,3/6/20,BR,Brazil,13.0,0.0,0.0,4.0,0.0,0.0,9.0,0.0,0.0,3.250000,NaN,NaN


In [19]:
# Compute Death Evolution per Day
target_country.loc[:, ('death_rate_previous_day')] = 0
death_rate_previous_day_column_ix = target_country.columns.get_loc('death_rate_previous_day')
totaldeaths_ix = target_country.columns.get_loc('totaldeaths')

for i in range(1, len(target_country.index)):
    target_country.iloc[i, death_rate_previous_day_column_ix] = \
        (target_country.iloc[i, totaldeaths_ix] / target_country.iloc[i-1, totaldeaths_ix])-1

NameError: name 'target_country' is not defined

In [ ]:
# Compute Death Percentage
target_country.loc[:, ('death_percentage')] = df.apply(lambda row: (np.divide(row['totaldeaths'], row['totalcases']) * 100), axis=1)